In [ ]:
from collections import OrderedDict

from dpipe.im.visualize import slice3d
import dpipe.im.metrics as metrics
import dpipe.io

from liver_localization import SelectSlice
from model import get_unet
import numpy as np

import os

from tensorboardX import SummaryWriter

import torchvision
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from functools import partial

In [22]:
def get_patches(length):
    for i in range(length//16):
        yield i*16, (i+1)*16-1
    yield(length-16,length-1)
for a,b in get_patches(89):
    print(a,b)

0 15
16 31
32 47
48 63
64 79
73 88


In [12]:
class LiverDataset(Dataset):
    def __init__(self, path="/home/egor/LITS/LITS_reoriented/", 
                 path_to_preds="/home/egor/LITS/liver_preds/",
                 min_val=-300., max_val=300.,train=True):
        
        self.indices = sorted([int(i[:-4]) for i in sorted(os.listdir(path_to_preds))])
        if train:
            self.indices = self.indices[:int(0.9*len(self.indices))]
        else:
            self.indices = self.indices[int(0.9*len(self.indices)):]
        self.path = path
        self.path_to_preds = path_to_preds
        self.transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), ])
        self.len = len(self.indices)
        
    def __getname__(self, idx):
        elem = self.indices[idx]
        
        if(os.path.exists(os.path.join(self.path,f"segmentation-{elem}.nii"))):
            seg_name = os.path.join(self.path,f"segmentation-{elem}.nii")
        else:
            seg_name = os.path.join(self.path,f"segmentation-{elem}_ras.nii")
        
        if(os.path.exists(os.path.join(self.path,f"volume-{elem}.nii"))):
            vol_name = os.path.join(self.path,f"volume-{elem}.nii")
        else:
            vol_name = os.path.join(self.path,f"volume-{elem}_ras.nii")
        
        return vol_name, seg_name
    
    def __getslice__(self, idx):
        elem = self.indices[idx]
        pred_name = os.path.join(self.path_to_preds, f"{elem}.npy")
        #print("__getslice__: ", pred_name, end='\t')
        pred = (dpipe.io.load(pred_name)>0.5)[0][0]
        _, (min_idx, max_idx) = SelectSlice(pred, return_indices=True)
        return (min_idx, max_idx)
        
    def scale_ct(self, x: np.ndarray, min_value: float = -300., max_value: float = 300.):
        x = np.float32(np.clip(x, a_min=min_value, a_max=max_value))
        x -= np.min(x)
        return x
        x /= np.max(x)
        
    def __getitem__(self, idx):
        #print(idx)
        min_idx, max_idx = self.__getslice__(idx)
        vol_name, seg_name = self.__getname__(idx)
        #print("__getitem__:", vol_name, seg_name)
        vol = self.scale_ct(dpipe.io.load(vol_name))
        seg = torch.tensor(dpipe.io.load(seg_name)==2,
                           dtype=torch.float32).permute(2,0,1)
        
        
        vol = self.transforms(vol)
        vol = vol.type(torch.float32)
        
        vol = vol[min_idx:max_idx,:,:]
        seg = seg[min_idx:max_idx,:,:]
        
        #print(vol.shape)
        #print(seg.shape)
        return vol.unsqueeze(dim=0), seg.unsqueeze(dim=0)
    def __len__(self):
        return self.len

NameError: name 'Dataset' is not defined

In [ ]:
writer = SummaryWriter(logdir="./logdir")

train_dataset = LiverDataset()
train_loader = DataLoader(train_dataset, shuffle=True, num_workers=4)

val_dataset = LiverDataset(train=False)
val_loader = DataLoader(val_dataset, num_workers=4)

RESULT_DIR = "./results"
if os.path.exists(RESULT_DIR)==False:
    os.mkdir(RESULT_DIR)

In [ ]:
model = get_unet(1,1)
_ = model.cuda()

In [ ]:
criterion = nn.BCELoss()

hist = {"train_loss": [],
        "val_loss": []}

train_len = len(train_loader)

In [ ]:
lr = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr)
num_epochs = 20

In [ ]:
for i in range(num_epochs):
    
    model.train()
    k_max = len(train_loader)
    for k, (img, mask) in enumerate(train_loader):  
        img = img.cuda()
        mask = mask.cuda()
        pred = model(img)
        loss = criterion(pred, mask)
        print("loss computed")
        writer.add_scalar("Train loss", loss.detach().item())
        loss.backward()
        optimizer.zero_grad()
        optimizer.step()
        
        pred_cpu = pred.detach().cpu().numpy()>0.5
        mask_cpu = mask.cpu().numpy() > 0.5
        hausdorff_distance = metrics.hausdorff_distance(pred_cpu, mask_cpu)
        dice_loss = metrics.dice_loss(pred_cpu, mask_cpu)
        writer.add_scalar("Train Hausdorff", hausdorff_distance)
        writer.add_scalar("Train dice loss", dice_loss)

        print(f"Epoch {i}/{num_epochs} iter {k}/{k_max}", end = "\r")
    
    torch.save(model.state_dict(), os.path.join(RESULT_DIR ,f"model_{i}.pt"))
    with torch.no_grad():
        model.eval()
        
        k_max = len(val_loader)
        for k, (img_layers, mask_layers) in enumerate(val_loader):  
            img = img.cuda()
            mask = mask.cuda()
            pred = model(img)
            loss = criterion(pred, mask)
            writer.add_scalar("Train loss", loss.item())
            
            pred_cpu = pred.detach().cpu().numpy()>0.5
            mask_cpu = mask.cpu().numpy() > 0.5
            hausdorff_distance = metrics.hausdorff_distance(pred_cpu, mask_cpu)
            dice_loss = metrics.dice_loss(pred_cpu, mask_cpu)
            writer.add_scalar("Val Hausdorff", hausdorff_distance)
            writer.add_scalar("val dice loss", dice_loss)
